<a href="https://colab.research.google.com/github/gudev-db/AI_Stuff/blob/main/SACvsPPOvsDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Comparação de métodos de RL para resolver Car Racing V3

In [ ]:
!pip install gymnasium torch matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 10.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!pip install Box2D

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 37.8 MB/s eta 0:00:00


In [ ]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.5 MB/s eta 0:00:00


##SAC

In [ ]:
import gymnasium as gym
from stable_baselines3 import SAC

# Create CarRacing environment
env = gym.make('CarRacing-v3')

# Initialize SAC
model = SAC('CnnPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=100000)

# Save the model
model.save("sac_car_racing")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -34.9    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 28       |
|    time_elapsed    | 141      |
|    total_timesteps | 4000     |
| train/             |          |
|    actor_loss      | -18      |
|    critic_loss     | 0.223    |
|    ent_coef        | 0.311    |
|    ent_coef_loss   | -5.9     |
|    learning_rate   | 0.0003   |
|    n_updates       | 3899     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -34.3    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 28       |
|    time_elapsed    | 283      |
|    total_tim

##DQN

In [ ]:
import gymnasium as gym
from stable_baselines3 import DQN

# Create CarRacing environment
env = gym.make('CarRacing-v3')

# Initialize DQN
model = DQN('CnnPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Save the model
model.save("dqn_car_racing")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.discrete.Discrete'>,) as action spaces but Box([-1.  0.  0.], 1.0, (3,), float32) was provided

##PPO

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
import torch

# Create CarRacing environment
env = gym.make('CarRacing-v3')

# Check if CUDA is available and use the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"



# Create PPO model with custom policy architecture, device setting, and hyperparameters
model = PPO(
    'CnnPolicy',
    env,
    verbose=0,
    learning_rate=1e-2,
    gamma=0.99,
    n_steps=2048,
    policy_kwargs={
        'net_arch': {'pi': [256, 256, 256,256], 'vf': [512, 512, 512,512]},  # Separate architecture for value network (vf)
    },
    clip_range=0.1,
    batch_size=2048
)

# Train the model
model.learn(total_timesteps=100000)

# Save the model
model.save("ppo_car_racing_PPO")



In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
import torch

# Create CarRacing environment
env = gym.make('CarRacing-v3')

# Check if CUDA is available and use the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"



# Create PPO model with custom policy architecture, device setting, and hyperparameters
model = PPO(
    'CnnPolicy',
    env,
    verbose=0,
    ent_coef=0.005,
    learning_rate=1e-2,
    gamma=0.99,
    n_steps=2048,
    policy_kwargs={
        'net_arch': {'pi': [256, 256, 256,256,256,256,256], 'vf': [512,512,512,512, 512, 512,512]},  # Separate architecture for value network (vf)
    },
    clip_range=0.1,
    batch_size=2048
)

# Train the model
model.learn(total_timesteps=100000)

# Save the model
model.save("ppo_car_racing_PPO2")


##Gerando video

In [ ]:
import gymnasium as gym
import numpy as np
import cv2
from stable_baselines3 import PPO
from IPython.display import HTML
from matplotlib import pyplot as plt
import matplotlib.animation as animation

# Load the trained model
model = PPO.load("ppo_car_racing_PPO")

# Create the environment
env = gym.make('CarRacing-v3')

# Set up the video capture
frames = []

# Run the trained agent in the environment
state, info = env.reset()
done = False

while not done:
    action, _states = model.predict(state)  # Get the action from the model
    state, reward, terminated, truncated, info = env.step(action)

    # Capture the frame from the environment
    frames.append(state)

    if terminated or truncated:
        break

# Convert the frames to a video format (use OpenCV to create a video)
video_filename = "car_racing_videoPPO.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_height, frame_width, _ = frames[0].shape
out = cv2.VideoWriter(video_filename, fourcc, 30.0, (frame_width, frame_height))

# Write frames to video
for frame in frames:
    out.write((frame * 255).astype(np.uint8))  # Convert to uint8 format for video
out.release()

# Display the video link
video_filename


'car_racing_videoPPO.avi'

In [ ]:
import gymnasium as gym
import numpy as np
import cv2
from stable_baselines3 import PPO
from IPython.display import HTML
from matplotlib import pyplot as plt
import matplotlib.animation as animation

# Carregar o modelo treinado
model = SAC.load("sac_car_racing")

# Criar o ambiente
env = gym.make('CarRacing-v3')

# Definir o número de passos ou episódios para rodar por mais tempo
num_steps = 10000000000000000  # Número de passos que o agente irá rodar
# Ou, para rodar vários episódios:
# num_episodes = 10  # Número de episódios que o agente irá rodar

# Listar os quadros do vídeo
frames = []

# Rodar o agente por um número fixo de passos
state, info = env.reset()

for _ in range(num_steps):
    action, _states = model.predict(state)  # Obter a ação do modelo
    state, reward, terminated, truncated, info = env.step(action)

    # Capturar o quadro do ambiente
    frames.append(state)

    if terminated or truncated:
        break  # Termina se o episódio terminar, mas você pode desconsiderar isso se rodar por um número fixo de passos

# Caso tenha escolhido rodar vários episódios, descomente o código abaixo
# for _ in range(num_episodes):
#     state, info = env.reset()
#     done = False
#
#     while not done:
#         action, _states = model.predict(state)  # Obter a ação do modelo
#         state, reward, terminated, truncated, info = env.step(action)
#
#         # Capturar o quadro do ambiente
#         frames.append(state)
#
#         if terminated or truncated:
#             break  # Termina o episódio

# Converter os quadros para um formato de vídeo (usar OpenCV para criar o vídeo)
video_filename = "car_racing_videoSAC.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codificador para o vídeo
frame_height, frame_width, _ = frames[0].shape
out = cv2.VideoWriter(video_filename, fourcc, 30.0, (frame_width, frame_height))  # 30 FPS

# Escrever os quadros no vídeo
for frame in frames:
    out.write((frame * 255).astype(np.uint8))  # Converter para o formato uint8 para o vídeo

# Liberar o escritor de vídeo
out.release()

# Exibir o caminho do vídeo gerado
video_filename


'car_racing_videoSAC.avi'

<img src='https://github.com/HiddenBeginner/study-notes/blob/main/contents/img/CartRacing-v2_DQN.gif?raw=true'></img>